In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler,RobustScaler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
from svmutil import *

In [3]:
import pandas as pd

In [5]:
data=pd.read_csv('tagged_data.csv')
data

In [4]:
# y=data['rating'].values.tolist()
# y

In [6]:
len(data.iloc[2,1:1001].values.tolist())

1000

In [7]:
x=[]
for i in range(16524):
    x.append(data.iloc[i,1:1001].values.tolist())

In [8]:
len(x)

16524

In [9]:
len(y)

16524

In [10]:
from imblearn.under_sampling import AllKNN
allknn = AllKNN(random_state=0)
rexim, reyim = allknn.fit_sample(x, y)

In [11]:
pd.value_counts(reyim)

4     11136
12      653
17      511
9       312
dtype: int64

In [12]:
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import InstanceHardnessThreshold
iht = InstanceHardnessThreshold(random_state=0,
                                estimator=LogisticRegression())
rexm1, reym1 = iht.fit_sample(rexim, reyim)

In [13]:
pd.value_counts(reym1)

17    312
12    312
9     312
4     312
dtype: int64

In [14]:
xm=rexm1.tolist()
ym=reym1.tolist()

In [32]:
dataout = pd.DataFrame(xm)
dataout['y']=ym
dataout.to_csv('outim.csv')

In [43]:
prob  = svm_problem(ym, xm, isKernel=True)
# param = svm_parameter('-c 128 -v 10 -g 0.5') 82
param = svm_parameter('-c 128 -v 10 -g 0.5')
m=svm_train(prob, param)

Cross Validation Accuracy = 83.0929%


In [39]:
from ctypes import *
from ctypes.util import find_library
from os import path
import sys

In [40]:
__file__='C:/Users/Scott Sun/Desktop/yiluRA/libsvm/libsvm-3.22/python'
dirname = path.dirname(path.abspath(__file__))
path.join(dirname, r'../windows/libsvm.dll')

'C:\\Users\\Scott Sun\\Desktop\\yiluRA\\libsvm\\libsvm-3.22\\../windows/libsvm.dll'

In [41]:
libsvm = CDLL('C:/Users/Scott Sun/Desktop/yiluRA/libsvm/libsvm-3.22/windows/libsvm.dll')
libsvm

<CDLL 'C:/Users/Scott Sun/Desktop/yiluRA/libsvm/libsvm-3.22/windows/libsvm.dll', handle 7ffdee450000 at 0x13a6e2af0b8>

In [42]:
try:
	dirname = path.dirname(path.abspath(__file__))
	if sys.platform == 'win32':
		libsvm = CDLL(path.join(dirname, r'windows/libsvm.dll'))
	else:
		libsvm = CDLL(path.join(dirname, '../libsvm.so.2'))
except:
# For unix the prefix 'lib' is not considered.
	if find_library('svm'):
		libsvm = CDLL(find_library('svm'))
	elif find_library('libsvm'):
		libsvm = CDLL(find_library('libsvm'))
	else:
		raise Exception('LIBSVM library not found.')

In [43]:
def svm_train1(arg1, arg2=None, arg3=None):
	"""
	svm_train(y, x [, options]) -> model | ACC | MSE
	svm_train(prob [, options]) -> model | ACC | MSE
	svm_train(prob, param) -> model | ACC| MSE

	Train an SVM model from data (y, x) or an svm_problem prob using
	'options' or an svm_parameter param.
	If '-v' is specified in 'options' (i.e., cross validation)
	either accuracy (ACC) or mean-squared error (MSE) is returned.
	options:
	    -s svm_type : set type of SVM (default 0)
	        0 -- C-SVC		(multi-class classification)
	        1 -- nu-SVC		(multi-class classification)
	        2 -- one-class SVM
	        3 -- epsilon-SVR	(regression)
	        4 -- nu-SVR		(regression)
	    -t kernel_type : set type of kernel function (default 2)
	        0 -- linear: u'*v
	        1 -- polynomial: (gamma*u'*v + coef0)^degree
	        2 -- radial basis function: exp(-gamma*|u-v|^2)
	        3 -- sigmoid: tanh(gamma*u'*v + coef0)
	        4 -- precomputed kernel (kernel values in training_set_file)
	    -d degree : set degree in kernel function (default 3)
	    -g gamma : set gamma in kernel function (default 1/num_features)
	    -r coef0 : set coef0 in kernel function (default 0)
	    -c cost : set the parameter C of C-SVC, epsilon-SVR, and nu-SVR (default 1)
	    -n nu : set the parameter nu of nu-SVC, one-class SVM, and nu-SVR (default 0.5)
	    -p epsilon : set the epsilon in loss function of epsilon-SVR (default 0.1)
	    -m cachesize : set cache memory size in MB (default 100)
	    -e epsilon : set tolerance of termination criterion (default 0.001)
	    -h shrinking : whether to use the shrinking heuristics, 0 or 1 (default 1)
	    -b probability_estimates : whether to train a SVC or SVR model for probability estimates, 0 or 1 (default 0)
	    -wi weight : set the parameter C of class i to weight*C, for C-SVC (default 1)
	    -v n: n-fold cross validation mode
	    -q : quiet mode (no outputs)
	"""
	prob, param = None, None
	if isinstance(arg1, (list, tuple)):
		assert isinstance(arg2, (list, tuple))
		y, x, options = arg1, arg2, arg3
		param = svm_parameter(options)
		prob = svm_problem(y, x, isKernel=(param.kernel_type == PRECOMPUTED))
	elif isinstance(arg1, svm_problem):
		prob = arg1
		if isinstance(arg2, svm_parameter):
			param = arg2
		else:
			param = svm_parameter(arg2)
	if prob == None or param == None:
		raise TypeError("Wrong types for the arguments")

	if param.kernel_type == PRECOMPUTED:
		for xi in prob.x_space:
			idx, val = xi[0].index, xi[0].value
			if xi[0].index != 0:
				raise ValueError('Wrong input format: first column must be 0:sample_serial_number')
			if val <= 0 or val > prob.n:
				raise ValueError('Wrong input format: sample_serial_number out of range')

	if param.gamma == 0 and prob.n > 0:
		param.gamma = 1.0 / prob.n
	libsvm.svm_set_print_string_function(param.print_func)
	err_msg = libsvm.svm_check_parameter(prob, param)
	if err_msg:
		raise ValueError('Error: %s' % err_msg)

	if param.cross_validation:
		l, nr_fold = prob.l, param.nr_fold
		target = (c_double * l)()
		libsvm.svm_cross_validation(prob, param, nr_fold, target)
		ACC, MSE, SCC = evaluations(prob.y[:l], target[:l])
		if param.svm_type in [EPSILON_SVR, NU_SVR]:
			print("Cross Validation Mean squared error = %g" % MSE)
			print("Cross Validation Squared correlation coefficient = %g" % SCC)
			return MSE
		else:
			print("Cross Validation Accuracy = %g%%" % ACC)
			print(target[:l])#revise
			return target[:l]#ACC
	else:
		m = libsvm.svm_train(prob, param)
		m = toPyModel(m)

		# If prob is destroyed, data including SVs pointed by m can remain.
		m.x_space = prob.x_space
		return m

In [80]:
type(m)

list

In [79]:
prob  = svm_problem(ym, xm, isKernel=True)
# param = svm_parameter('-c 128 -v 10 -g 0.5') 82
param = svm_parameter('-c 32 -v 10 -g 2')
m=svm_train1(prob, param)

Cross Validation Accuracy = 83.0929%
[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.

In [137]:
X_train, X_test, y_train, y_test = train_test_split(xm,ym,test_size=0.1,random_state=0,shuffle=True)

In [138]:
prob  = svm_problem(y_train, X_train, isKernel=True)
# param = svm_parameter('-c 128 -v 10 -g 0.5') 82
param = svm_parameter('-c 128 -g 0.5 -b 1')
modelm1=svm_train(prob, param)

In [126]:
# svm_get_nr_class(modelm1)

In [139]:
p_label, p_acc, p_val = svm_predict(y_test, X_test, modelm1,'-b 1')#c=6

Accuracy = 43.2% (54/125) (classification)


In [140]:
# p_label=[]
# for i in X_test:
#     label = libsvm.svm_predict_probability(modelm1,i,4)#c=6
#     p_label.append(label)

In [141]:
# p_acc
# p_val
# p_label

In [46]:
prob  = svm_problem(y[:13500], x[:13500], isKernel=True)
param = svm_parameter('-c 32 -v 10')
m=svm_train(prob, param)

Cross Validation Accuracy = 79.5556%


In [45]:
p_label, p_acc, p_val = svm_predict(y[13501:16523], x[13501:16523], m)#c=6

Accuracy = 73.3289% (2215/3022) (classification)


In [48]:
prob  = svm_problem(y, x, isKernel=True)
param = svm_parameter('-c 36 -v 10')
m=svm_train(prob, param)

Cross Validation Accuracy = 78.4132%


In [49]:
prob  = svm_problem(y, x, isKernel=True)
param = svm_parameter('-c 32 -v 10 -g 0.5')
m=svm_train(prob, param)

Cross Validation Accuracy = 79.9383%


In [50]:
prob  = svm_problem(y, x, isKernel=True)
param = svm_parameter('-c 32 -v 10 -g 2')
m1=svm_train(prob, param)

Cross Validation Accuracy = 80.3558%


In [58]:
prob  = svm_problem(y, x, isKernel=True)
param1 = svm_parameter('-c 32 -g 2')
model_80=svm_train(prob, param1)

In [56]:
type(y)

list

In [66]:
import numpy as np
import random
random.random()--EVAN

0.5912063975856594

In [59]:
svm_save_model('model_80', model_80)

In [67]:
type(p_label)

list

In [ ]:
# p_val

# Text Only

In [69]:
data2 = pd.read_csv('out10.csv')
data2.head()

,index,Description,name,party,parties,boyfriend,girlfriend,virtual,wild,castle,...,hot,island,night,real,no real,spice,split,swim,trainer,rating
0,0,"1812 Cannon Defense Borodino', 'Full 3D Gamepl...",1812,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
1,1,Special Battle of Gettysburg Edition of Cannon...,1863,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
2,2,CHALLENGE your creativity and imagination in t...,! Hip Hop Fashion Stars Dress Up,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,3,!Timer 2 is the ultimate timer for your Mac. S...,!Timer,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
4,4,"Apocalypse is here! ', 'All humans are dead or...",#01,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12


In [70]:
y2=data2['rating']
y2.value_counts()

4     12957
12     1796
9      1260
17      511
Name: rating, dtype: int64

In [24]:
data3 = pd.read_csv('out13.csv')
data3 = data3.dropna()
x2=data3.drop('rating',axis=1)
y2=data3['rating']

In [88]:
sm = SMOTE(random_state=42)
x_res1, y_res1 = sm.fit_sample(rex1, rey1)

In [90]:
pd.value_counts(y_res1)

17    508
12    508
9     508
4     508
dtype: int64

In [29]:
# x2=[]
# for i in range(len(data2['rating'])):
#     x2.append(data2.iloc[i,3:15].values.tolist())
# x2[0]
x3=rext1.tolist()
y3=reyt1.tolist()

In [68]:
#12clos
prob  = svm_problem(y3, tx3.tolist(), isKernel=True)
param = svm_parameter('-c 512 -v 10 -g 0.0078125')
modelt1=svm_train(prob, param)

Cross Validation Accuracy = 77.9186%


In [96]:
prob  = svm_problem(y3, x3, isKernel=True)
param = svm_parameter('-c 256 -v 10 -g 0.005')
modelt1=svm_train(prob, param)

Cross Validation Accuracy = 88.5496%


In [106]:
prob  = svm_problem(y3, x3, isKernel=True)
param = svm_parameter('-c 128 -v 10 -g 0.005')
# param = svm_parameter('-c 128 -v 10 -g 0.005') 88.7405
modelt1=svm_train(prob, param)

Cross Validation Accuracy = 88.5496%


In [110]:
prob  = svm_problem(y3, x3, isKernel=True)
param = svm_parameter('-c 64 -v 10 -g 0.005')
# param = svm_parameter('-c 128 -v 10 -g 0.005') 97.4549
modelt1=svm_train(prob, param)

Cross Validation Accuracy = 97.581%


In [30]:
prob  = svm_problem(y3, x3, isKernel=True)
param = svm_parameter('-c 64 -g 0.005')
# param = svm_parameter('-c 128 -v 10 -g 0.005') 97.4549
modelt=svm_train(prob, param)

In [31]:
p_label, p_acc, p_val = svm_predict(y3, x3, modelt)#c=6

Accuracy = 28.4987% (448/1572) (classification)


output

In [30]:
dataout = pd.DataFrame(x4t)
dataout['y']=y4
dataout.to_csv('t2.csv')

In [31]:
dataout = pd.DataFrame(x4m)
dataout['y']=y4
dataout.to_csv('m2.csv')

Text+Image

In [5]:
data4 = pd.read_csv('out14.csv')
data4 = data4.dropna()
# xcom=data4.drop('rating',axis=1)
# ycom=data4['rating']
label=[]
for i in data4['rating']:
    if i==4:
        label.append(0)
    else:
        label.append(1)
data4['label']=label
data4

,rating,party,parties,boyfriend,girlfriend,virtual,wild,castle,dorm,roommate,...,bubble,cheeseburger,"parallel bars, bars","flagpole, flagstaff",coffee mug,"rubber eraser, rubber, pencil eraser",stole,carbonara,dumbbell,label
0,12,0,0,0,0,0,0,0,0,0,...,0.000151,0.000179,0.000095,0.000185,0.003662,0.000437,0.000201,0.000047,0.000112,1
1,12,0,0,0,0,0,0,0,0,0,...,0.000119,0.000056,0.000133,0.000217,0.000475,0.008665,0.000730,0.000061,0.000296,1
2,4,0,0,0,0,0,0,0,0,0,...,0.000475,0.000385,0.000153,0.000312,0.000730,0.002258,0.003329,0.000113,0.000893,0
3,4,0,0,0,0,0,0,0,0,0,...,0.000220,0.000093,0.000083,0.000084,0.000113,0.000141,0.000223,0.000048,0.000075,0
4,12,0,0,0,0,0,0,0,0,0,...,0.000615,0.000269,0.000088,0.000517,0.000435,0.000194,0.000371,0.000169,0.000174,1
5,4,0,0,0,0,0,0,0,0,1,...,0.000143,0.000148,0.000191,0.000160,0.003465,0.003103,0.001299,0.000060,0.000102,0
6,4,0,0,0,0,0,0,0,0,0,...,0.000481,0.000264,0.000128,0.000247,0.000482,0.000467,0.000178,0.000132,0.000181,0
7,12,0,0,0,0,0,0,0,0,0,...,0.000127,0.000143,0.000052,0.000167,0.000438,0.000434,0.000051,0.000127,0.000087,1
8,4,0,0,0,0,0,0,0,0,0,...,0.000394,0.049727,0.000118,0.000499,0.019242,0.003114,0.000464,0.000287,0.000161,0
9,4,0,0,0,0,0,0,0,0,0,...,0.000490,0.000378,0.000074,0.000232,0.011714,0.001234,0.000258,0.000258,0.000169,0


In [6]:
xcl=data4.drop('label',axis=1)
ycl=data4['label']

In [7]:
xc2=data4.drop('rating',axis=1)
yc2=data4['rating']

In [ ]:
from imblearn.under_sampling import AllKNN
allknn = AllKNN(random_state=0)
rexcl, reycl = allknn.fit_sample(xc2, yc2)

In [25]:
pd.value_counts(reyc)

4.0     3383
12.0     888
9.0      690
17.0     508
dtype: int64

In [24]:
rexct = x4t=rex1[:,1:1316]
reyct = rex1[:,0]
from imblearn.under_sampling import NeighbourhoodCleaningRule
ncr = NeighbourhoodCleaningRule(random_state=0)
rexc, reyc = ncr.fit_sample(rexct, reyct)

In [11]:
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import InstanceHardnessThreshold
iht = InstanceHardnessThreshold(random_state=0,
                                estimator=LogisticRegression())
rex1, rey1 = iht.fit_sample(xcl, ycl)

In [26]:
datac = pd.DataFrame(rex1)
datacc = pd.DataFrame(rexc)

In [27]:
x4t=rexc[:,0:315].tolist()
x4m=rexc[:,316:1315].tolist()
y4 = reyc.tolist()

In [100]:
x4tl=x4t.tolist()
x4ml=x4m.tolist()
y4=rey1.tolist()

In [11]:
prob  = svm_problem(y4, x4, isKernel=True)
param = svm_parameter('-c 64 -v 10 -g 0.005')
# param = svm_parameter('-c 128 -v 10 -g 0.005') 97.4549
modelti1=svm_train(prob, param)

Cross Validation Accuracy = 97.6374%


In [73]:
prob  = svm_problem(y4, x4t, isKernel=True)
param = svm_parameter('-c 8 -v 10 -g 0.0078125')
# param = svm_parameter('-c 128 -v 10 -g 0.005') 97.4549
modelti=svm_train(prob, param)

Cross Validation Accuracy = 88.2794%
[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.

In [61]:
prob  = svm_problem(y4, x4m, isKernel=True)
param = svm_parameter('-c 32 -g 2')
modelmi1=svm_train(prob, param)

In [62]:
svm_save_model('tm.model', modelti1)

In [63]:
svm_save_model('imm.model', modelmi1)

In [64]:
datag=pd.read_csv('output-im+t.csv')
datag

,Content_rating,"kit fox, Vulpes macrotis",English setter,Siberian husky,Australian terrier,"English springer, English springer spaniel","grey whale, gray whale, devilfish, Eschrichtius gibbosus, Eschrichtius robustus","lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens",Egyptian cat,"ibex, Capra ibex",...,bid,spit,baseball.1,transform,VIOLENT,vodka,storms,story,puzzle,hunter
0,4,0.000167,0.000163,0.000076,0.000187,0.000116,0.000122,0.000132,0.000130,0.000145,...,0,0,0,0,0,0,0,0,0,0
1,4,0.000145,0.000178,0.000183,0.000228,0.000211,0.000166,0.000163,0.000330,0.000396,...,0,0,0,0,0,0,0,0,0,0
2,4,0.000193,0.000472,0.000086,0.000051,0.000220,0.000054,0.000091,0.000208,0.000124,...,0,0,0,0,0,0,0,0,-1,0
3,13,0.000073,0.000065,0.000047,0.000082,0.000092,0.000050,0.000046,0.000131,0.000059,...,0,0,0,0,0,0,0,0,0,0
4,4,0.000190,0.000104,0.000088,0.000143,0.000200,0.000085,0.000105,0.000259,0.000463,...,0,0,0,0,0,0,0,0,-1,0
5,4,0.000146,0.000176,0.000101,0.000454,0.000081,0.000061,0.000163,0.000118,0.000101,...,0,0,0,0,0,0,0,0,0,0
6,4,0.000120,0.000103,0.000098,0.000268,0.000108,0.000235,0.000216,0.000109,0.000094,...,0,0,0,0,0,0,0,0,0,0
7,4,0.000297,0.000132,0.000183,0.000270,0.000265,0.000235,0.000227,0.000131,0.000174,...,0,0,0,0,0,0,0,0,0,0
8,4,0.000271,0.000141,0.000188,0.000141,0.000198,0.000181,0.000269,0.000198,0.000208,...,0,0,0,0,0,0,0,0,0,0
9,4,0.000210,0.000124,0.000203,0.000222,0.000232,0.000281,0.000216,0.000249,0.000186,...,0,0,0,0,0,0,0,0,0,0


In [67]:
y=datag['Content_rating'].values.tolist()
for i in range(len(y)):
    if y[i]==10:
        y[i]=9
    if y[i]==13:
        y[i]=12
xm=[]
for i in range(28774):
    xm.append(datag.iloc[i,1:1001].values.tolist())
xt=[]
for i in range(28774):
    xt.append(datag.iloc[i,1003:1324].values.tolist())

In [68]:
labelm, accm, valm = svm_predict(y,xm,modelmi1)

Accuracy = 29.9541% (8619/28774) (classification)


In [70]:
labelt, acct, valt = svm_predict(y,xt,modelti1)

Accuracy = 24.1086% (6937/28774) (classification)


In [72]:
outlab = datag[['Content_rating','Description','Item_name']]
outlab['labelm']=labelm
outlab['labelt']=labelt
outlab.to_csv('outlab2.csv')

C:\Users\Scott Sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Scott Sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


logistic regression

In [32]:
import numpy as np
import matplotlib.pyplot as plt

# 使用交叉验证的方法，把数据集分为训练集合测试集
from sklearn.model_selection import train_test_split

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split,cross_val_score

C:\Users\Scott Sun\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [55]:
logx = []
for i in range(len(modelmi1)):
    temp=[]
    temp.append(modelmi1[i])
    temp.append(modelti1[i])
    logx.append(temp)
len(logx)

5469

In [56]:
X_trainl, X_testl, y_trainl, y_testl = train_test_split(logx,y4,test_size=0.1,random_state=42)

In [57]:
cls = LogisticRegression(multi_class='multinomial',solver='lbfgs',max_iter=1000)
cls.fit(X_trainl, y_trainl)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [58]:
cls.score(X_testl, y_testl)

0.8318098720292505

In [59]:
scores=cross_val_score(cls,logx,y4,cv=10)
# precisions = cross_val_score(cls,logx,y4,cv=10, scoring=
# 'precision')
# recall = cross_val_score(cls,logx,y4,cv=10, scoring=
# 'recall')

In [38]:
scores

array([ 0.80109489,  0.81021898,  0.80474453,  0.79890311,  0.83363803,
        0.84277879,  0.80255941,  0.80987203,  0.81651376,  0.84036697])

In [12]:
prob  = svm_problem(y4, x4, isKernel=True)
param = svm_parameter('-c 64 -g 0.005')
modelti_1=svm_train(prob, param)

In [13]:
svm_save_model('model_97', modelti_1)

Standard Scaler

In [82]:
scaler = StandardScaler()
scaler.fit(x3)
tx3 = scaler.transform(x3)
# scaler.fit([y3])
# ty3 = scaler.transform([y3])
# ty3[0].tolist()
# tx3

prototype generation

In [31]:
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=0)
X_res2, y_res2 = cc.fit_sample(x2, y2)

prototype selection

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0, replacement=True)
X_resampled, y_resampled = rus.fit_sample(X, y)

In [13]:
from imblearn.under_sampling import NearMiss
nm1 = NearMiss(random_state=0, version=1)
X_res1, y_res1 = nm1.fit_sample(x2, y2)

edit

In [77]:
from imblearn.under_sampling import RepeatedEditedNearestNeighbours
renn = RepeatedEditedNearestNeighbours(random_state=42)
rex, rey = renn.fit_sample(x2, y2)

In [46]:
from imblearn.under_sampling import EditedNearestNeighbours
enn = EditedNearestNeighbours(random_state=42)
rex, rey = enn.fit_sample(x2, y2)

In [25]:
from imblearn.under_sampling import AllKNN
allknn = AllKNN(random_state=0)
rext, reyt = allknn.fit_sample(x2, y2)

In [107]:
from imblearn.under_sampling import NeighbourhoodCleaningRule
ncr = NeighbourhoodCleaningRule(random_state=0)
rex1, rey1 = ncr.fit_sample(rex, rey)

In [27]:
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import InstanceHardnessThreshold
iht = InstanceHardnessThreshold(random_state=0,
                                estimator=LogisticRegression())
rext1, reyt1 = iht.fit_sample(rext, reyt)

In [ ]:
pd.value_counts(reyt1)

In [ ]:
12957/